# 기출 9회_작업형1

## 문제1

1. 총대출액은 신용대출과 담보대출의 합이다.
2. 각 지역코드 내에서, 성별(예: 1과 2)별로 '총대출액'의 합계를 계산하시오.
3. 성별 간 총대출액의 차이가 가장 큰 지역코드를 구하시오.

In [61]:
# My1) groupby, unstack

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/loan.csv")

df['총대출액'] = df['신용대출']+df['담보대출']
df = df.groupby(['지역코드', '성별'])['총대출액'].sum()
df = df.unstack()
df['diff'] = abs(df[1]-df[2])
df = df.sort_values('diff', ascending=False)
print(df.index[0])
# print(df)

4100000278


In [90]:
# My2) pivot_table

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/loan.csv")

df['총대출액'] = df['신용대출']+df['담보대출']
# print(df)

grouped = df.pivot_table(index='지역코드', columns='성별', values='총대출액', aggfunc='sum')
grouped['diff'] = abs(grouped[1]-grouped[2])
print(grouped['diff'].idxmax())

4100000278


In [59]:
# 방법1

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/loan.csv")

# 1) 총대출액 컬럼 생성
df['총대출액'] = df['신용대출'] + df['담보대출']

# 2) 지역코드와 성별별로 총대출액 합계 계산
grouped = df.groupby(['지역코드', '성별'])['총대출액'].sum().unstack()

# 3) 성별 간 총대출액 차이가 가장 큰 지역코드 찾기 (절대값 사용)
grouped['차이'] = abs(grouped[1] - grouped[2])
result = grouped['차이'].idxmax()
print(result)

4100000278


In [58]:
# 방법2

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/loan.csv")

# 1) 총대출액 컬럼 생성
df['총대출액'] = df['신용대출'] + df['담보대출']

# 2) 지역코드와 성별별로 총대출액 합계 계산
grouped = df.pivot_table(index='지역코드', columns='성별', values='총대출액', aggfunc='sum')
# print(grouped)

# 3) 성별 간 총대출액 차이가 가장 큰 지역코드 찾기 (절대값 사용)
grouped['차이'] = abs(grouped[1] - grouped[2])
result = grouped['차이'].idxmax()
print(result)

4100000278


## 문제2

1. 검거율은 검거건수 / 발생건수이다.
2. 각 연도에서 가장 높은 검거율을 기록한 범죄유형을 찾으시오.
3. 해당 범죄유형의 검거건수를 구한 후, 이 값들을 모두 합산하시오. (정수로 구하시오)

In [108]:
# 방법1 

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/crime.csv")
# print(df)

# 1) "발생건수"와 "검거건수"를 따로 분리
cond1 = df['구분'] == '검거건수'
df1 = df[cond1].iloc[:, 2:].reset_index(drop=True)
# print(df1)
# print(df1.shape)

cond2 = df['구분'] == '발생건수'
df2 = df[cond2].iloc[:, 2:].reset_index(drop=True)
# print(df2)
# print(df2.shape)

# 2) 검거율 계산 (검거건수 / 발생건수)
# index가 매칭되어야 df끼리 계산이 가능함.
df3 = df1 / df2
# print(df3)

# 3) 각 연도에서 검거율이 가장 높은 범죄유형 찾기
listbox = df3.idxmax(axis=1)
# print(listbox)

# 4) 가장 높은 검거율을 기록한 범죄유형의 검거건수 가져오기
result = 0
for index, item in enumerate(listbox):
    result += df1.loc[index, item]

print(result)

7799


In [144]:
# 방법2

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/crime.csv")
print('df =============')
print(df)

# 1. 데이터를 길게 변환
df_long = pd.melt(df, id_vars=['연도', '구분'], var_name='범죄유형', value_name='건수')
print('df_long =============')
print(df_long)

# 2. 피벗(pivot)하여 발생건수와 검거건수를 한 행에 모으기
df_pivot = df_long.pivot_table(index=['연도', '범죄유형'], columns='구분', values='건수').reset_index()

# 3. 검거율 계산
df_pivot['검거율'] = df_pivot['검거건수'] / df_pivot['발생건수']
print('df_pivot =============')
print(df_pivot)

# 4. 각 연도별로 검거율이 가장 높은 범죄유형 찾기
listbox = df_pivot.groupby(['연도'])['검거율'].idxmax()
max_crime_df = df_pivot.loc[listbox]
print(max_crime_df)

# 5. 각 연도별 최고 검거율 범죄유형의 검거건수를 모두 합산하기
# result = 0
# for index in listbox:
#     result += df_pivot.loc[index, '검거건수']
# print(result)

result = max_crime_df['검거건수'].sum()
print(result)

df =============
      연도    구분  강력범죄  절도범죄  폭력범죄  지능범죄  풍속범죄  특별경제범죄  마약범죄  보건범죄  환경범죄  교통범죄  \
0   2014  발생건수   800   900   750   650   700    1034   600   850   800   900   
1   2014  검거건수   500   600   400   450   400     961   350   500   450   550   
2   2015  발생건수   900   768   800   750   700    1000   850   800   900   950   
3   2015  검거건수   600   713   500   550   400     600   500   450   550   600   
4   2016  발생건수   850   800   880   750   700     950   900   800   850   900   
5   2016  검거건수   500   600   812   550   400     600   500   450   500   550   
6   2017  발생건수  1000   900   850   800   750     950  1400   900  1000   950   
7   2017  검거건수   600   500   500   550   400     600  1300   500   550   600   
8   2018  발생건수  1450   900   850   800   750     950   900   800   850   900   
9   2018  검거건수  1350   600   500   550   400     600   500   450   500   550   
10  2019  발생건수  1000   900   850   800   750     950   900  1400  1000   950   
11  2019  검거건수   600   

## 문제3

1. 만족도의 결측치는 만족도의 평균값으로 대체한다.
2. 근속연수의 결측치는 동일한 부서 및 성과등급에 해당하는 직원들의 평균 근속연수로 대체한다. (이때, 평균 근속연수는 소수점 이하를 절사하여 정수로 변환하여 사용)
3. 각 직원의 '연봉 / 근속연수' 값을 계산하여, 그 값이 세 번째로 큰 직원의 근속연수를 구하시오.
4. 각 직원의 '연봉 / 만족도' 값을 계산하여, 그 값이 두 번째로 큰 직원의 교육참가횟수를 구하시오.
5. (3)과 (4)의 값을 더한 결과를 구하시오. (정수로 구하시오)

In [169]:
# 방법1

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/hr.csv")
print('df ============')
print(df)

# 1) 만족도 결측치 처리
# print(df['만족도'].isna().sum())
m = df['만족도'].mean()
df['만족도'] = df['만족도'].fillna(m)
# print(df['만족도'].isna().sum())

# 2) 근속연수 결측치 처리 (부서와 성과등급 기준 평균값으로 채움)
# print(df['근속연수'].isna().sum())
gm = df.groupby(['부서', '성과등급'])['근속연수'].transform('mean')
gm = gm.astype(int)
print('gm ============')
print(gm)
df['근속연수'] = df['근속연수'].fillna(gm)
# print(df['근속연수'].isna().sum())
# print(df)

# 3) 연봉 / 근속연수 계산 후 세 번째로 높은 사람의 근속연수 (A)
df['연봉_근속연수'] = df['연봉'] / df['근속연수']
df_year = df.nlargest(3, '연봉_근속연수')
A = df_year.iloc[-1]['근속연수']
print('df_year ============')
print(df_year)
print(A)

# 4) 연봉 / 만족도 계산 후 두 번째로 높은 사람의 교육참가횟수 (B)
# print(df)
df['연봉_만족도'] = df['연봉'] / df['만족도']
df_cnt = df.nlargest(2, '연봉_만족도')
B = df_cnt.iloc[-1]['교육참가횟수']
print('df_cnt ============')
print(df_cnt)
print(B)

# 5) 결과 출력
print('정답:', int(A+B))

df ============
      사원번호         부서 성과등급         연봉  근속연수  교육참가횟수  만족도
0    E0001  Marketing    B   73200000  14.0       5  6.0
1    E0002         IR    A   95100000   3.0       4  9.0
2    E0003    Manager    B   44700000   5.0       2  8.0
3    E0004         IR    B  122100000  16.0       3  8.0
4    E0005         IR    C   66000000   NaN       4  8.0
..     ...        ...  ...        ...   ...     ...  ...
995  E0996    Finance    B   90400000  15.0       3  2.0
996  E0997         IR    C  147600000  12.0       5  5.0
997  E0998      Sales    C   95600000   4.0       4  9.0
998  E0999      Sales    C   94400000   8.0       6  1.0
999  E1000         IR    A   92400000   9.0       2  5.0

[1000 rows x 7 columns]
gm ============
0      11
1      11
2      10
3      10
4      10
       ..
995    10
996    10
997    11
998    11
999    11
Name: 근속연수, Length: 1000, dtype: int64
df_year ============
      사원번호       부서 성과등급         연봉  근속연수  교육참가횟수       만족도      연봉_근속연수
357  E0358  Fina

In [ ]:
# 방법2
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/inf/refs/heads/main/p4/9_1/hr.csv")

# 1) 만족도 결측치 처리
# print(df['만족도'].isnull().sum())
m = df['만족도'].mean()
df['만족도'] = df['만족도'].fillna(m)
# print(df['만족도'].isnull().sum())

# 2) 근속연수 결측치 처리 (부서와 성과등급 기준 평균값으로 채움)
# print(df['근속연수'].isnull().sum())
gm = df.groupby(['부서', '성과등급'])['근속연수'].transform('mean')
gm = gm.astype(int)
df['근속연수'] = df['근속연수'].fillna(gm)
# print(df['근속연수'].isnull().sum())

# 3) 연봉 / 근속연수 계산 후 세 번째로 높은 사람의 근속연수 (A)
df['연봉_근속연수'] = df['연봉'] / df['근속연수']
df_year = df.sort_values('연봉_근속연수', ascending=False)
A = df_year.iloc[2]['근속연수']

# 4) 연봉 / 만족도 계산 후 두 번째로 높은 사람의 교육참가횟수 (B)
df['연봉_만족도'] = df['연봉'] / df['만족도']
df_like = df.sort_values('연봉_만족도', ascending=False)
B = df_like.iloc[1]['교육참가횟수']

# 5) 결과 출력
result = A + B
print(result)

7.0
